# PROGRAMA DEFINITIVO DE PAGE-SPEED

- Ya esta acabado, solo hay que pasarle la lista con todas las URL's a la funcion Global

- Devuelve un dataframe donde cada linea es una pagina, las variables tienen _m para móvil y _o para ordenador, 
  y también se printan los links que han fallado, estos se podrían volver a revisar con un time.sleep
  mas alto, o comprobar manualmente si estan en PageSpeed
  
- Hay dos módulos Global, el último es para la ejecución en paralelo, pero esta el riesgo de que bloqueen la IP y   el programa de error, si ocurre, utilizar el otro modulo para ejecución normal. (En la prueba con 10 link funcionaba bien)



In [1]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as ec
import requests
import time
from bs4 import BeautifulSoup
import pandas as pd

headers = {'user_agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.6.1 Safari/605.1.15'}
def INICIAR_DRIVER():
    options=Options()
    userAgent= 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.6.1 Safari/605.1.15'
    options.add_argument(f'user-agent={userAgent}')
    options.add_argument('--start-maximized')
    options.add_argument('--disable-web-security')
    options.add_argument('--disable-extensions')
    options.add_argument('--disable-notifications')
    options.add_argument('--ignore-certificate-errors')
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--allow-running-insecure-content')
    options.add_argument('--no-default-browser-check')
    options.add_argument('--no-first-run')
    options.add_argument('--no-proxy-server')
    options.add_argument('--disable-blink-features=AutomationControlled')
    exp_opt=['enable-automation', 'ignore-certificate-errors','enable-logging']
    options.add_experimental_option('excludeSwitches', exp_opt)
    prefs={'profile.default_content_setting_values.notifications':2, 'intl.accept_languages':['es-ES','es'],'credentials_enable_service':False}
    options.add_experimental_option('prefs',prefs)
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)
    return driver

In [2]:
def pagespeed(url, tm, numero):
    
    # Si no se ejecuta en paralelo quitar estas dos lineas de abajo:

    driver = INICIAR_DRIVER()
    wait = WebDriverWait(driver, 1.5)    
    path ='https://pagespeed.web.dev'
    driver.get(path)

    text_input = wait.until(ec.visibility_of_element_located((By.XPATH,'//*[@id="i4"]')))
    text_input.send_keys(url)
    buton = wait.until(ec.element_to_be_clickable((By.XPATH,'//*[@id="yDmH0d"]/c-wiz/div[2]/div/div[2]/form/div[2]/button')))
    buton.click()
    
    time.sleep(tm)
    
    res = driver.page_source
    soup = BeautifulSoup(res,'html.parser')
    
    driver.quit()
    
    try:
        soups = soup.find_all('div', role='tabpanel')
        
        columnas = ['Link','Tipo','LCP','INP','FCP','TTF','CLS','FID']
        disp = ['Móvil','Ordenador']
        df = pd.DataFrame(columns=columnas)
    
        
        for i,soup in enumerate(soups):
            dic = {'Link':url,'Tipo':disp[i],'LCP':pd.NA,'INP':pd.NA,'FCP':pd.NA,'TTF':pd.NA,'CLS':pd.NA,'FID':pd.NA}
            soup = soup.find('div', style = 'display: none;')
            elementos = soup.find_all('div', class_ = 'PZeSbe')
            for j in elementos:
                var = j.find('span').text
                var = var.split('(')[1][:3]
                value = j.find('div', class_='cOTkPe').text
                if len(value)> 40:
                    value = value.split('.')[2:]
                    value = str(value[0])+'.'+str(value[1])
                    dic[var]= value
                else:
                    dic[var]=value
                    
            df2 = pd.DataFrame([dic])
            df= pd.concat([df, df2], ignore_index=True)
        
        print(f'{GREEN}Hecho Link {numero}{RESET}')
        return df
    except:
        print(f'{RED}Error en {url}{RESET}')

In [3]:
# ESTE NO ES EN PARALELO, 3 VECES MÁS LENTO, HABRÍA QUE QUITAR LO QUE ESTA COMENTADO EN LA FUNCIÓN ANTERIOR

def Global(lista, tm):
    
    columnas = ['Link','Tipo','LCP','INP','FCP','TTF','CLS','FID']
    df = pd.DataFrame(columns=columnas)
    

    for i,j in enumerate(lista):
        try:
            df2 = pagespeed(j, tm, i)
            df = pd.concat([df, df2], ignore_index=True)
        except:
            print(f'{RED}Error con el link: {i}{RESET}')

    df_pivot = df.pivot(index='Link', columns='Tipo', values=['LCP', 'INP', 'FCP', 'TTF', 'CLS', 'FID'])
    df_pivot.columns = [f'{metric}_{device[0].lower()}' for metric, device in df_pivot.columns]
    df_pivot.reset_index(inplace=True)
    

    return df_pivot

In [4]:
# PRINCIPAL

def main():

    urls = []
    with open('/Users/pablogandia/Downloads/urls.txt','r') as f:
        for line in f:
            link = line.rstrip()
            urls.append(link)

    tm = 6

    print('EMPEZANDO PROGRAMA:')
    print('Hay ',len(urls),' links para scrapear')
    print()

    DF = Global(urls, tm)

    print()
    print('SCRAPPING FINALIZADO')

    DF = DF.dropna()

    errores = []

    for link in urls:
        if link not in list(DF['Link']):
            errores.append(link)

    if len(errores) != 0:
        tm = 12
        print()
        print('INICIANDO REVISIÓN')
        print()
        print('SE VAN A SCRAPEAR LOS LINKS QUE NO FUNCIONARON AUMENTANDO EL TIEMPO DE ESPERA')
        print()
        DF2 = Global(errores, tm)
        print('REVISIÓN FINALIZADA')

        DF_fin = pd.concat([DF, DF2], ignore_index=True)
        DF_fin.reset_index(inplace=True)

        error_fin = []
        for link in urls:
            if link not in list(DF_fin['Link']):
                error_fin.append(link)
                
        final = DF_fin

    else:
        final = DF

    print()
    print('LA BASE DE DATOS HA SIDO CREADA')
    print()

    if len(error_fin) > 0:
        print('Por algun error no aparecen en la base final los links: ')
        print()
        print('-'*30)
        print()
        for i in error_fin:
            print('- ',i)
    print()
    
    return final

In [5]:
RED = "\033[31m"
GREEN = "\033[32m"
RESET = "\033[0m"

DF = main()
DF.to_csv('/Users/pablogandia/Desktop/PageSpeed.csv')

EMPEZANDO PROGRAMA:
Hay  91  links para scrapear

Hecho Link 0
Hecho Link 1
Hecho Link 2
Hecho Link 3
Hecho Link 4
Hecho Link 5
Hecho Link 6
Hecho Link 7
Hecho Link 8
Hecho Link 9
Hecho Link 10
Hecho Link 11
Hecho Link 12
Hecho Link 13
Hecho Link 14
Hecho Link 15
Hecho Link 16
Hecho Link 17
Hecho Link 18
Hecho Link 19
Hecho Link 20
Hecho Link 21
Hecho Link 22
Hecho Link 23
Hecho Link 24
Hecho Link 25
Hecho Link 26
Hecho Link 27
Hecho Link 28
Hecho Link 29
Hecho Link 30
Hecho Link 31
Hecho Link 32
Hecho Link 33
Hecho Link 34
Hecho Link 35
Hecho Link 36
Hecho Link 37
Hecho Link 38
Hecho Link 39
Hecho Link 40
Hecho Link 41
Hecho Link 42
Hecho Link 43
Hecho Link 44
Hecho Link 45
Hecho Link 46
Hecho Link 47
Hecho Link 48
Hecho Link 49
Error en https://www.eurosport.com/
Hecho Link 51
Hecho Link 52
Hecho Link 53
Hecho Link 54
Hecho Link 55
Hecho Link 56
Hecho Link 57
Hecho Link 58
Hecho Link 59
Hecho Link 60
Hecho Link 61
Hecho Link 62
Hecho Link 63
Hecho Link 64
Hecho Link 65
Hecho Link 66
